# Paris Metro Project

## Rules :

The goal is to pass in one day by all the metro stations in Paris. You have to go at least once (though not necessarily one) by each of the Paris metro stations intramural, during a day.

Authorized transportations are: the metro (however, the use of the bus, a car or a taxi, or even a bicycle, is prohibited). It is not necessary to browse the metro stations located outside the limits of Paris (it stops at stations with names in "porte ... .."); BUT it is not forbidden to leave Paris (eg. by going to the Boulogne Jean Jaurès station, which has an interest). 
If a station is crossed by several lines, it is not necessary (but not prohibited) to go as many times as there are lines that pass by. There is no need to stop at each station: cross enough. 
Finally, two stations are considered different if they have different names (including Châtelet is not the same station as Les Halles).

In [1]:
# Import packages
import numpy as np
import pandas as pd
import networkx as nx

### 1) Load datas in two separate DataFrames

In [2]:
# Load the vertices txt file & put it in a DataFrame
File = open("C:/Users/Thomas/Desktop/My Files/DSTI/COURSES/(WEEK 12) Metaheuristics Optimisation/metro_vertices.txt", "r")
vertices = pd.DataFrame()
for line in File:
    a = line.split(' ', 1)
    index = int(a[0])
    vertices.loc[index,'Station'] = a[1].strip()
File.close()

In [3]:
# Load the edges txt file & put it in a DataFrame
edges = pd.read_csv('C:/Users/Thomas/Desktop/My Files/DSTI/COURSES/(WEEK 12) Metaheuristics Optimisation/metro_edges.txt', sep=' ', header = None)
# Rename columns
edges.columns = ['origin', 'destination', 'time']
# Change each column type to be integer type
edges = edges.astype(int)

### 2) Data cleaning

In [4]:
# Create a list of useless stations with corresponding IDs
# Which means stations outside Paris & without change
# These stations are without interest for our exercise
UselessStations = [37, 253, 25, 193, 178, 81, 72, 188, 187, 363, 364, 365, 161, 237, 179, 167, 53, 372, 186, 185,
                   89, 90, 91, 328, 43, 66, 301, 88, 181, 114, 183, 28, 29, 374, 134, 152, 108, 10, 319, 15, 318,
                   48, 182, 126, 180, 112, 251, 4, 171, 166, 252, 102, 130]

In [5]:
# Create an empty list "ind"
ind = []
# Append to this list each index where there is a useless station for the DataFrame "edges"
for index, row in edges.iterrows():
    if row['origin'] in UselessStations:
        ind.append(index)
    elif row['destination'] in UselessStations:
        ind.append(index)
# Remove the useless rows
edges = edges.drop(ind)

In [6]:
# Create an empty list "ind"
ind = []
# Append to this list each index where there is a useless station for the DataFrame "vertices"
for index, row in vertices.iterrows():
    if index in UselessStations:
        ind.append(index)
# Remove the useless rows
Useful = vertices.drop(ind)
# Remove duplicates to know the number of distinct stations we have to pass by
Useful = Useful.drop_duplicates(subset = 'Station')
DistinctStations = len(Useful)
print("We have to pass by", str(DistinctStations), "distinct stations !")

We have to pass by 243 distinct stations !


### 3) Solving the problem with Dijkstra

In [7]:
# Create a list with all IDs except the useless ones
ID = list(set(range(376)) - set(UselessStations))
# Create a dictionnary to store the results
dic = {}

In [8]:
# Release memory by deleting useless variables
del(UselessStations, ind)

In [9]:
# Create a graph thanks to the networkx library
graph = nx.Graph()

In [10]:
# Iterate over the dataframe "edges" to add the edges to the graph
for index, row in edges.iterrows():
    graph.add_edge(row[0], row[1], weight=row[2])

In [11]:
# Initiate a for loop to test the algorithm starting by each ID
for i in ID:
    # Create 3 variables to store time, IDs and stations names
    RouteTime = 0
    RouteID = []
    RouteName = []
    # Append the ID of the initial station to the RouteID list
    RouteID.append(i)
    # Append the name of the initial station to the RouteName list
    RouteName.append(vertices.iloc[i]['Station'])
    # Initiate a while loop that stops when the algo passed by 243 distinct stations
    while (len(set(RouteName)) < DistinctStations):
        # Using the single_source_dijkstra function to get length & path 
        # of the last visited station
        length, path = nx.single_source_dijkstra(graph, source=RouteID[-1])
        # Iterate over the length dictionnary until we met an ID not contained in RouteID
        for key, value in length.items():
            if key not in RouteID:
                # Get the station name associated to the ID
                StationName = vertices.iloc[key]['Station']
                # Delete the first element of the path list 
                # as this ID has already been appended in the previous loop
                del(path[key][0])
                # Iterate over the path list
                for i in path[key]:
                    # Append each ID to the Route ID list
                    RouteID.append(i)
                    # Append each station name to the RouteName list
                    RouteName.append(vertices.iloc[i]['Station'])
                # Add time to the RouteTime variable
                RouteTime += value
                break
    # Update the dictionnary to store the results
    dic.update({RouteTime : RouteName})

### 4) Results

In [12]:
# Find the best time 
MinTime = min(dic.keys())
print('The minimum time to pass by each station in Paris at least one time is'
      , MinTime, 'sec /', MinTime/(60*60), 'hours')

The minimum time to pass by each station in Paris at least one time is 22206 sec / 6.168333333333333 hours


In [13]:
# Find the path associated to the best time
BestPath = dic[MinTime]
BestPath

['Charles Michels',
 'Avenue Émile Zola',
 'La Motte Picquet, Grenelle',
 'Ségur',
 'Duroc',
 'Vaneau',
 'Sèvres Babylone',
 'Mabillon',
 'Odéon',
 'Cluny, La Sorbonne',
 'Maubert Mutualité',
 'Cardinal Lemoine',
 'Jussieu',
 "Gare d'Austerlitz",
 "Gare d'Austerlitz",
 'Quai de la Rapée',
 'Bastille',
 'Bréguet-Sabin',
 'Richard Lenoir',
 'Oberkampf',
 'République',
 'Jacques Bonsergent',
 "Gare de l'Est",
 'Gare du Nord',
 'Stalingrad',
 'Jaurès',
 'Laumière',
 'Ourcq',
 'Porte de Pantin',
 'Ourcq',
 'Laumière',
 'Jaurès',
 'Jaurès',
 'Stalingrad',
 'La Chapelle',
 'Barbès Rochechouart',
 'Anvers',
 'Pigalle',
 'Blanche',
 'Place de Clichy',
 'Rome',
 'Villiers',
 'Monceau',
 'Courcelles',
 'Ternes',
 'Charles de Gaulle, Étoile',
 'Victor Hugo',
 'Porte Dauphine',
 'Victor Hugo',
 'Charles de Gaulle, Étoile',
 'Charles de Gaulle, Étoile',
 'Argentine',
 'Porte Maillot',
 'Argentine',
 'Charles de Gaulle, Étoile',
 'George V',
 'Franklin D. Roosevelt',
 'Champs Élysées, Clémenceau',
 '